## 0.导入所需的库

In [ ]:
#selenium模块用于网页自动化
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
#time模块用于爬虫间的休眠，避免频繁访问
import time
#数据处理常用的pandas、numpy
import pandas as pd
import numpy as np

## 1.使用selenium驱动浏览器进行网页自动化-part1

In [ ]:
import time
from selenium.webdriver.common.keys import Keys

url = r'https://exam.sac.net.cn/pages/registration/sac-publicity-report.html'

driver = webdriver.Chrome()
driver.get(url)

def construct_url(a):
    return r'https://exam.sac.net.cn/pages/registration/sac-publicity-finish.html?aoiId=' + a

def catch_id(a):
    return a.get_attribute('onclick').split('(')[1][:-1]

#按6下空格，确保网页滑动到最底端，页面上内容全部加载完毕
for i in range(6):
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(1)

all_companies = driver.find_elements_by_xpath('//*[@id="publicityOtherList"]/tr/td[2]/a')
all_companies_names = [a.text for a in all_companies]
all_companies_urls = [construct_url(catch_id(a)) for a in all_companies]
print(all_companies_urls)

driver.quit()

## 2.使用pandas整理数据

In [ ]:
import pandas as pd

#companies_df用来存储公司的基本信息
companies_df = pd.DataFrame(columns=['name','url'])
companies_df['name'] = all_companies_names
companies_df['url'] = all_companies_urls
companies_df

## 3.使用selenium驱动浏览器进行网页自动化-part2

In [ ]:
#analysts_df用来存储分析师的基本信息
analysts_df = pd.DataFrame(columns=['company','name','url'])

count = 0
for i in range(len(all_companies_urls)):
    company_url = all_companies_urls[i]
    company_name = all_companies_names[i]
    print(company_name)
    print(company_url)
    driver = webdriver.Chrome()
    driver.get(company_url)
    time.sleep(5)
    
    
    #要进行翻页操作，首先确定要翻多少次页
    page_number = int(driver.find_element_by_xpath('//*[@id="sp_1"]').text)
    
    for pn in range(page_number-1): #进行page_number次翻页

        #同样的通过按空格键，跳转到页面底部，保证页面内容加载完成
        for send_key in range(6):
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(1)  

        analysts_urls = driver.find_elements_by_xpath('/html/body/table/tbody/tr[2]/td/table/tbody/tr[2]/td/div/div/div[3]/div[4]/div/table/tbody/tr/td[1]')
        analysts_urls = [a.get_attribute('title') for a in analysts_urls]
        analysts_urls = analysts_urls[1:]
        
        analysts_names = driver.find_elements_by_xpath('/html/body/table/tbody/tr[2]/td/table/tbody/tr[2]/td/div/div/div[3]/div[4]/div/table/tbody/tr/td[2]/a')
        analysts_names = [a.text for a in analysts_names]
        
        #存储每个公司的，每个分析师的个人主页地址，存储到analysts_df中
        for ii in range(len(analysts_names)):
            company_name = company_name
            name = analysts_names[ii]
            url = analysts_urls[ii]
            
            analysts_df.loc[count,:] = [company_name,name,url]
            count += 1
        
        #找到下一页的按钮，点击
        next_button = driver.find_element_by_xpath('//*[@id="next_t"]/span')
        next_button.click()
        
driver.quit()

## 4.下载每位分析师的图片

In [ ]:
#download pics

#建立pics文件夹用来存储下载的图片
if os.path.exists('./pics') == False:
    os.mkdir('./pics')
    
driver = webdriver.Chrome()
for url in analysts_df['url']:
    #进入每个分析师的个人主页，定位其照片元素，找到对应url
    driver.get('https://exam.sac.net.cn/pages/registration/sac-finish-person.html?r2SS_IFjjk='+url)
    img = driver.find_element_by_xpath('//*[@id="RPI_PHOTO_PATH"]/img')
    img_url = img.get_attribute('src')
    
    print(img_url)
    
    #使用requests模块访问并下载图片
    r = requests.get(img_url)
    with open('./pics/%s.png' % url, 'wb') as f:
        f.write(r.content)    